In [ ]:
!export AUTOGRAPH_VERBOSITY=0
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow as tf 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# tf.compat.v1.logging.set_verbosity(100)

In [ ]:
%load_ext autoreload
%autoreload 2
import cantera as ct
import os
import gym 
import numpy as np 
import pandas as pd
from stable_baselines.common.policies import MlpPolicy, CnnLstmPolicy,MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common import set_global_seeds
from stable_baselines import ACKTR, PPO2
from envs.sim_env import SimEnv, NO_idx, CO_idx, O2_idx, H2O_idx
from envs.SimUtils import solvePhi_airSplit, equil, runMainBurner, correctNOx
import seaborn as sns

In [ ]:
model = PPO2.load('Trained Models/PPO2_MlpPolicy_11172019_1000000.zip')

In [4]:
from stable_baselines.common.vec_env import DummyVecEnv
sim_env = SimEnv()
env = DummyVecEnv([lambda: sim_env])
# env = SimEnv()
obs = env.reset()
env.action_space.high

array([17733.37587108,  1690.62375258,  9642.49689204])

In [5]:
import pdb 
game_over = False
results_list = []
while not game_over: 
    action, _states = model.predict(obs)
#     action = env.action_space.high[np.newaxis,:]
    obs, rewards, game_over, info = env.step(action)
    env.render(mode='human')
    g = env.envs[0].sec_stage_gas
    results_list.append([
        env.envs[0].age/1e-3, 
        g.T, 
        correctNOx(g.X[NO_idx], g.X[H2O_idx], g.X[O2_idx]),
        correctNOx(g.X[CO_idx], g.X[H2O_idx], g.X[O2_idx]),
        env.envs[0].mfc_main.mdot(0),
        env.envs[0].mfc_fuel_sec.mdot(0),
        env.envs[0].mfc_air_sec.mdot(0), 
        env.envs[0].remaining_main_burner_mass,
        env.envs[0].sec_fuel_remaining,
        env.envs[0].sec_air_remaining,
        rewards, ])

step|age_(ms)|T|phi_norm|NO|CO|Rem_Main|Rem_SecFuel|Rem_SecAir|Mdot_Main|Mdot_SecFuel|Mdot_SecAir|Max_Main|Max_SecFuel|Max_SecAir|Reward|Reward_Reactants|Reward_T|Reward_NO|Reward_CO
1| 0.00| 1451.17| -0.00| 0.06| 1.08| 88.67| 1.69| 9.64| 3.92| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 223.24|0.00| 23.75|199.49|0.00
2| 0.00| 1469.79| -0.00| 0.06| 1.17| 88.67| 1.69| 9.64| 2.59| 0.00| 0.00| 17733.37| 1690.62| 9642.50| 224.02|0.00| 24.53|199.49|0.00
3| 0.00| 1469.60| 0.07| 0.06| 1.41| 88.67| 1.69| 9.64| 3.60| 0.03| 0.00| 17733.37| 1690.62| 9642.50| 224.01|0.00| 24.52|199.49|0.00
4| 0.00| 1476.66| 0.06| 0.07| 1.58| 88.67| 1.69| 9.64| 3.26| 0.00| 0.00| 17733.37| 1690.62| 9642.50| 224.31|0.00| 24.82|199.49|0.00
5| 0.00| 1343.29| 0.56| 0.05| 1.29| 88.67| 1.69| 9.64| 3.43| 0.74| 0.00| 17733.37| 1690.62| 9642.50| 219.07|0.00| 19.58|199.49|0.00
6| 0.01| 1362.04| 0.52| 0.06| 1.31| 88.67| 1.69| 9.64| 2.74| 0.00| 0.00| 17733.37| 1690.62| 9642.50| 219.76|0.00| 20.26|199.49|0.00
7| 0.01| 1376.93| 0.49|

67| 0.07| 989.73| 0.88| 0.03| 0.53| 88.67| 1.69| 9.64| 3.35| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.30|0.00| 9.80|199.50|0.00
68| 0.07| 993.75| 0.88| 0.03| 0.54| 88.67| 1.69| 9.64| 3.35| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.38|0.00| 9.88|199.50|0.00
69| 0.07| 996.61| 0.88| 0.03| 0.54| 88.67| 1.69| 9.64| 2.41| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.44|0.00| 9.94|199.50|0.00
70| 0.07| 1000.33| 0.88| 0.03| 0.54| 88.67| 1.69| 9.64| 3.19| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.51|0.00| 10.01|199.50|0.00
71| 0.07| 1003.68| 0.88| 0.03| 0.55| 88.67| 1.69| 9.64| 2.92| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.58|0.00| 10.08|199.50|0.00
72| 0.07| 1007.27| 0.87| 0.03| 0.55| 88.67| 1.69| 9.64| 3.16| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.66|0.00| 10.16|199.50|0.00
73| 0.07| 1010.92| 0.87| 0.03| 0.55| 88.67| 1.69| 9.64| 3.28| 0.00| 0.00| 17733.33| 1690.57| 9642.50| 209.73|0.00| 10.23|199.50|0.00
74| 0.07| 1014.68| 0.87| 0.03| 0.56| 88.67| 1.69| 9.64| 3.42| 0.00| 0.00| 1

142| 0.14| 990.66| 0.88| 0.03| 0.49| 88.67| 1.69| 9.64| 3.46| 0.53| 0.00| 17733.28| 1690.50| 9642.48| 209.31|0.00| 9.82|199.50|0.00
143| 0.14| 992.12| 0.88| 0.03| 0.49| 88.67| 1.69| 9.64| 2.57| 0.00| 0.00| 17733.28| 1690.50| 9642.48| 209.34|0.00| 9.84|199.50|0.00
144| 0.14| 993.81| 0.87| 0.03| 0.50| 88.67| 1.69| 9.64| 3.01| 0.00| 0.00| 17733.28| 1690.50| 9642.48| 209.38|0.00| 9.88|199.50|0.00
145| 0.14| 995.94| 0.87| 0.03| 0.50| 88.67| 1.69| 9.64| 3.82| 0.00| 0.00| 17733.28| 1690.50| 9642.48| 209.42|0.00| 9.92|199.50|0.00
146| 0.15| 997.76| 0.87| 0.03| 0.50| 88.67| 1.69| 9.64| 3.29| 0.00| 0.00| 17733.28| 1690.50| 9642.48| 209.46|0.00| 9.96|199.50|0.00
147| 0.15| 999.93| 0.87| 0.03| 0.50| 88.67| 1.69| 9.64| 3.95| 0.00| 0.00| 17733.28| 1690.50| 9642.48| 209.50|0.00| 10.00|199.50|0.00
148| 0.15| 989.99| 0.88| 0.03| 0.49| 88.67| 1.69| 9.64| 2.83| 5.62| 0.00| 17733.28| 1690.50| 9642.48| 209.30|0.00| 9.80|199.50|0.00
149| 0.15| 991.91| 0.88| 0.03| 0.49| 88.67| 1.69| 9.64| 3.55| 0.00| 0.00| 1

208| 0.21| 947.26| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 3.96| 5.47| 0.00| 17733.24| 1690.41| 9642.48| 208.46|0.00| 8.97|199.50|0.00
209| 0.21| 948.72| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 3.75| 0.00| 0.00| 17733.24| 1690.41| 9642.48| 208.49|0.00| 9.00|199.50|0.00
210| 0.21| 949.80| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 2.80| 0.00| 0.00| 17733.24| 1690.41| 9642.48| 208.51|0.00| 9.02|199.50|0.00
211| 0.21| 951.03| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 3.19| 0.00| 0.00| 17733.24| 1690.41| 9642.48| 208.53|0.00| 9.04|199.50|0.00
212| 0.21| 951.07| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 3.31| 1.02| 0.00| 17733.24| 1690.41| 9642.48| 208.53|0.00| 9.04|199.50|0.00
213| 0.21| 952.15| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 2.83| 0.00| 0.00| 17733.24| 1690.41| 9642.48| 208.55|0.00| 9.06|199.50|0.00
214| 0.21| 953.06| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 2.40| 0.00| 0.00| 17733.24| 1690.41| 9642.48| 208.57|0.00| 9.08|199.50|0.00
215| 0.21| 954.24| 0.89| 0.03| 0.42| 88.67| 1.69| 9.64| 3.12| 0.00| 0.00| 17

281| 0.28| 970.02| 0.88| 0.03| 0.41| 88.67| 1.69| 9.64| 2.95| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.88|0.00| 9.41|199.50|0.00
282| 0.28| 970.65| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 2.18| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.90|0.00| 9.42|199.50|0.00
283| 0.28| 971.64| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 3.45| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.91|0.00| 9.44|199.50|0.00
284| 0.28| 972.44| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 2.79| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.93|0.00| 9.45|199.50|0.00
285| 0.29| 973.36| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 3.25| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.95|0.00| 9.47|199.50|0.00
286| 0.29| 974.41| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 3.69| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.97|0.00| 9.49|199.50|0.00
287| 0.29| 975.37| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 3.40| 0.00| 0.00| 17733.19| 1690.36| 9642.47| 208.99|0.00| 9.51|199.50|0.00
288| 0.29| 976.09| 0.88| 0.03| 0.42| 88.67| 1.69| 9.64| 4.04| 0.42| 0.00| 17

...

15722| 15.72| 991.72| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.98| 0.00| 0.00| 17723.13| 1677.01| 9641.04| -3676.85|0.00| 9.84|199.50|0.00
15723| 15.72| 991.70| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.84| 2.14| 0.00| 17723.13| 1677.01| 9641.04| -3677.59|0.00| 9.84|199.50|0.00
15724| 15.72| 991.71| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.28| 0.00| 0.00| 17723.13| 1677.01| 9641.04| -3678.33|0.00| 9.84|199.50|0.00
15725| 15.72| 991.73| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.67| 0.00| 0.00| 17723.13| 1677.01| 9641.04| -3679.07|0.00| 9.84|199.50|0.00
15726| 15.73| 991.74| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.24| 0.00| 0.00| 17723.13| 1677.01| 9641.04| -3679.81|0.00| 9.84|199.50|0.00
15727| 15.73| 991.76| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.57| 0.00| 0.00| 17723.13| 1677.01| 9641.04| -3680.55|0.00| 9.84|199.50|0.00
15728| 15.73| 991.71| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.87| 3.12| 0.00| 17723.12| 1677.01| 9641.04| -3681.30|0.00| 9.84|199.50|0.00
15729| 15.73| 991.73| 0.88| 0.02| 0.88| 8

15790| 15.79| 991.16| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 2.69| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3727.50|0.00| 9.83|199.50|0.00
15791| 15.79| 991.17| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.28| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3728.25|0.00| 9.83|199.50|0.00
15792| 15.79| 991.19| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.29| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3729.00|0.00| 9.83|199.50|0.00
15793| 15.79| 991.21| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.29| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3729.74|0.00| 9.83|199.50|0.00
15794| 15.79| 991.23| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.70| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3730.49|0.00| 9.83|199.50|0.00
15795| 15.79| 991.24| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.78| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3731.24|0.00| 9.83|199.50|0.00
15796| 15.80| 991.26| 0.88| 0.02| 0.88| 88.62| 1.68| 9.64| 3.78| 0.00| 0.00| 17723.08| 1676.92| 9641.04| -3731.99|0.00| 9.83|199.50|0.00
15797| 15.80| 991.21| 0.88| 0.02| 0.88| 8

15856| 15.86| 991.57| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 2.55| 0.00| 0.00| 17723.04| 1676.89| 9641.04| -3777.06|0.00| 9.83|199.50|0.00
15857| 15.86| 991.59| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 3.04| 0.00| 0.00| 17723.04| 1676.89| 9641.04| -3777.82|0.00| 9.83|199.50|0.00
15858| 15.86| 991.61| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 3.99| 0.00| 0.00| 17723.04| 1676.89| 9641.04| -3778.57|0.00| 9.83|199.50|0.00
15859| 15.86| 991.50| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 3.39| 6.60| 0.00| 17723.04| 1676.88| 9641.04| -3779.33|0.00| 9.83|199.50|0.00
15860| 15.86| 991.44| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 3.08| 3.99| 0.00| 17723.04| 1676.88| 9641.04| -3780.09|0.00| 9.83|199.50|0.00
15861| 15.86| 991.46| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 3.17| 0.00| 0.00| 17723.04| 1676.88| 9641.04| -3780.84|0.00| 9.83|199.50|0.00
15862| 15.86| 991.47| 0.88| 0.02| 0.89| 88.62| 1.68| 9.64| 2.74| 0.00| 0.00| 17723.04| 1676.88| 9641.04| -3781.59|0.00| 9.83|199.50|0.00
15863| 15.86| 991.49| 0.88| 0.02| 0.89| 8

15923| 15.92| 991.48| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.51| 0.00| 0.00| 17723.00| 1676.82| 9641.04| -3827.81|0.00| 9.83|199.50|0.00
15924| 15.92| 991.50| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.46| 0.00| 0.00| 17723.00| 1676.82| 9641.04| -3828.57|0.00| 9.83|199.50|0.00
15925| 15.92| 991.52| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.40| 0.00| 0.00| 17723.00| 1676.82| 9641.04| -3829.34|0.00| 9.83|199.50|0.00
15926| 15.93| 991.53| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.83| 0.62| 0.00| 17723.00| 1676.82| 9641.04| -3830.10|0.00| 9.83|199.50|0.00
15927| 15.93| 991.55| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.76| 0.00| 0.00| 17723.00| 1676.82| 9641.04| -3830.86|0.00| 9.83|199.50|0.00
15928| 15.93| 991.56| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.56| 0.00| 0.00| 17722.99| 1676.82| 9641.04| -3831.62|0.00| 9.83|199.50|0.00
15929| 15.93| 991.58| 0.88| 0.02| 0.89| 88.61| 1.68| 9.64| 3.71| 0.00| 0.00| 17722.99| 1676.82| 9641.04| -3832.38|0.00| 9.83|199.50|0.00
15930| 15.93| 991.44| 0.88| 0.02| 0.89| 8

15992| 15.99| 991.74| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 3.32| 0.00| 0.00| 17722.95| 1676.78| 9641.04| -3880.52|0.00| 9.84|199.50|0.00
15993| 15.99| 991.75| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 2.82| 0.00| 0.00| 17722.95| 1676.78| 9641.04| -3881.29|0.00| 9.84|199.50|0.00
15994| 15.99| 991.77| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 2.64| 0.00| 0.00| 17722.95| 1676.78| 9641.04| -3882.05|0.00| 9.84|199.50|0.00
15995| 15.99| 991.73| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 3.02| 0.00| 17.08| 17722.95| 1676.78| 9641.02| -3882.82|0.00| 9.84|199.50|0.00
15996| 16.00| 991.75| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 3.70| 0.00| 0.00| 17722.95| 1676.78| 9641.02| -3883.59|0.00| 9.84|199.50|0.00
15997| 16.00| 991.77| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 3.28| 0.00| 0.00| 17722.95| 1676.78| 9641.02| -3884.36|0.00| 9.84|199.50|0.00
15998| 16.00| 991.78| 0.88| 0.02| 0.90| 88.61| 1.68| 9.64| 2.99| 0.00| 0.00| 17722.95| 1676.78| 9641.02| -3885.12|0.00| 9.84|199.50|0.00
15999| 16.00| 991.80| 0.88| 0.02| 0.90| 

In [ ]:
action, _states = model.predict(obs)

In [ ]:
df = pd.DataFrame(data=np.vstack(results_list), columns=['age', 'T', 'NO', 'CO', 'mdot_main', 'mdot_fuel_sec', 'mdot_air_sec', 'rem_main', 'rem_fuel_sec', 'rem_air_sec', 'reward'])
df = df.iloc[0:-1]
df.to_csv("simResults_500000.csv", index=False)

In [ ]:
df

In [ ]:
df['mdot_main'].hist()

In [ ]:
df.plot('age', 'mdot_fuel_sec')

In [ ]:
env.envs[0].action_space.high[2]